In [ ]:
#이 프로그램에서 Ensemble 테스트는 각 모델별 결과를 이어받는 형태로 기존 방식으로 구현
#신규방식인 각 모델 결과를 모아서 최적의 영역을 선택 추출하는 방법은 new_RetinaNet - ENSEMBLE_TEST - with_anno.ipynb로 구현

import os
import pandas as pd
from itertools import groupby
from itertools import chain
from operator import eq
import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import shutil
import tensorflow as tf

%matplotlib inline
%load_ext autoreload
%autoreload 2

OS_ENV = 'LINUX' # 'LINUX' or 'WIN'
MODEL_ENV = 'ensemble' # 'ensemble' or 'separate'
DATE = '20181112'
DATA_TO_TEST = 'abn_all' # 'abn1' or 'abn2' or 'abn_all' or 'normal' or 'normal_1'
#normal_1': #2018. 1월말 데이터중 Normal 1318 file

#SCORE_VALUE_LIST = [0.5,0.4]
SCORE_VALUE_LIST = [0.4]

In [ ]:
if OS_ENV == 'LINUX':
    result_save_base_path = '/home/huray/workspace/han_work/han_test/results/'
else:
    result_save_base_path = 'D:/Work/NCC/han_test/results/'
result_save_subpath = os.path.join(result_save_base_path, DATE + '_' + MODEL_ENV, DATA_TO_TEST)
result_save_path = result_save_subpath
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

########## 2D list에서 특정 값이 위치한 index return ####################
def find(searchList, elem):
    for ix, row in enumerate(searchList):
        for iy, i in enumerate(row):
            if i==elem:
                #print('{},{}'.format(ix,iy))
                return [ix,iy]
    return []    

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [ ]:
model_list = []
#resnet50
#model_list.append('snapshots/1001_resnet50_ep100_with_valid/resnet50_csv_84.h5')
#model_list.append('snapshots/1001_resnet50_ep100_with_valid/1017_snapshot_resnet50_csv_90_ep100_with_valid/resnet50_csv_03.h5')
#model_list.append('snapshots/1001_resnet50_ep100_with_valid/1017_snapshot_resnet50_csv_90_ep100_with_valid/resnet50_csv_19.h5')
#model_list.append('snapshots/1001_resnet50_ep100_with_valid/1017_snapshot_resnet50_csv_90_ep100_with_valid/resnet50_csv_27.h5')
model_list.append('snapshots/1001_resnet50_ep100_with_valid/1017_snapshot_resnet50_csv_90_ep100_with_valid/resnet50_csv_36.h5')

#resnet101
#model_list.append('snapshots/0928_resnet101_ep100_with_valid/resnet101_csv_61.h5')
model_list.append('snapshots/0928_resnet101_ep100_with_valid/resnet101_csv_59.h5')
###model_list.append('snapshots/0928_resnet101_ep100_with_valid//1030_snapshot_resnet101_csv_61_ep100_with_valid/resnet101_csv_35.h5')
#model_list.append('snapshots/0928_resnet101_ep100_with_valid//1030_snapshot_resnet101_csv_61_ep100_with_valid/resnet101_csv_29.h5')
#model_list.append('snapshots/0928_resnet101_ep100_with_valid//1030_snapshot_resnet101_csv_61_ep100_with_valid/resnet101_csv_56.h5')

#resnet152
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/resnet152_csv_74.h5')
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_32.h5')
model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_45.h5')
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_53.h5')
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_57.h5')
#model_list.append('snapshots/0927_resnet152_ep100_with_valid/1012_snapshot_resnet152_csv_74_ep100_with_valid/resnet152_csv_84.h5')


####################################### 이전모델 ###################################
#ensemble 아래 e44 + e29
#single e29
#model_list.append('snapshots/0326_more_aug/resnet101_csv_44.h5')   
#model_list.append('snapshots/0525_add_ncc_mass_data_strong_aug/resnet101_csv_29.h5')
#######

#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_25.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_29.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_47.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_48.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_49.h5')



## Initialize data generators

In [ ]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

if DATA_TO_TEST == 'abn1':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn1.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn1_win.csv'
elif DATA_TO_TEST == 'abn2':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn2.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn2_win.csv'
elif DATA_TO_TEST == 'normal':
    if OS_ENV == 'LINUX':    
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_normal.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_normal_win.csv'
elif DATA_TO_TEST == 'normal_1': #2018. 1월말 데이터중 Normal 1381 file
    if OS_ENV == 'LINUX':    
        data_csv_path = '/home/huray/data/NCC_trainset/img_retinanet/data_ncc_normal_180130.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC_trainset/img_retinanet/data_ncc_normal_180130.csv'                
elif DATA_TO_TEST == 'abn_all':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn_merged.csv'
        #data_csv_path = '/home/huray/data/NCC/img_retinanet/0914_new_NCC_test_only_abn.csv'
        #data_csv_path = '/home/huray/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_imsi.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn_merged_win.csv'        
        #data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_anno_for_win.csv'
        #data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_for_win.csv'        
else:
    print('WRONG DATA CSV PATH!')
    raise
    
if OS_ENV == 'LINUX':
    label_csv_path = '/home/huray/data/new_trainset/class_2_malig_or_norm.csv'
else:
    label_csv_path = 'D:/Work/NCC/data/new_trainset/class_2_malig_or_norm.csv'    
    

# create a generator for testing data
val_generator = CSVGenerator(
    data_csv_path,
    label_csv_path,
    transform_generator=val_image_data_generator,
    batch_size=1
)

########## csv로 부터 테스트 파일목록을 read ######
test_file_df = pd.read_csv(data_csv_path, header=None)
num_of_test_files = len(test_file_df)
print('total test file : {}'.format(num_of_test_files))

########### 테스트파일 Sort ######
test_file_list = list(test_file_df[0]) #csv에서 읽어온 테스트 이미지파일 정보 목록
test_file_list.sort(key=lambda x: '-'.join(x.split('-')[:2])) #테스트 파일 목록을 groupby 하기 위해서 정렬

########### annotaion이 있는 테스트 파일정보를 dictionary 형태로 구성 ##########
#파일명을 key로 하여 그룹화딘 목록의 위치를 value로 매칭하여 구성
anno_list = list(test_file_df[1])
annotation_dict = {}
for f_idx, f_name in enumerate(test_file_list):
    #print('{},{},{}'.format(f_idx,f_name,anno_list[f_idx]))
    if not np.isnan(anno_list[f_idx]): #annotation 값이 없는것은 nan 값으로 표시되어 이를 판단
        annotation_dict[f_name] = []

#print(len(annotation_dict))        
#print(annotation_dict)

## define and initialize

In [ ]:
number_abnormal_detect_from_normal = 0 #normal 파일에서 abnormal로 검출된 경우의 수
#img_index_list = list(range(0, num_of_test_files))
not_detected_img_index_list = list(range(0, num_of_test_files))
#print(len(not_detected_img_index_list))

ordinal=lambda x:["1st","2nd","3rd","4th","5th","6th"][x-1]

############# for 테스트 파일목록 Groupping ############
groupby_key_list = [] #그룹명(테스트 파일 단위 그룹 명) 목록
groupby_file_list = [] #그룹화된 테스트파일 목록
groupby_detected_number_for_files = [] #검출(detect)된 수에 대한 그룹화된 목록(그룹화된 테스트 목록 구조와 동일하게 매핑)
groupby_detected_info_for_anno_files = [] #annotation이 있는 파일에 대한 검출(detect)여부정보 (그룹화된 테스트 목록 구조와 동일하게 매핑)
#list_not_detected_group_index = [] #검출되지 않은 그룹의 index 목록 

#M_max_scores = [] #abnormal 테스트 파일에서 detect된 anchor score 중 가장 높은 score
#N_max_scores = [] #normal 각 테스트 파일에서 detect된 anchor score 중 가장 높은 score

#테스트 파일 목록을 group by하여 list에 저장
#groupby key(그룹핑 기준(단위)는 파일명을 '-'로 split 한뒤 앞쪽 2번째 요소까지 추출함)
for key, group in groupby(test_file_list, key=lambda x: '-'.join(x.split('-')[:2])):
    #그룹 명(key) 목록
    groupby_key_list.append(key) 
    temp = list(group)
    
    #그룹화된 전체 테스트파일 목록 
    groupby_file_list.append(temp) 
    
    #그룹화 된 테스트 파일별 검출정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    groupby_detected_number_for_files.append([0]*len(temp))
    
    #annotation 정보가 있는 그룹화 된 테스트 파일별 검출정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    groupby_detected_info_for_anno_files.append([0]*len(temp))

    #그룹화 된 테스트 파일별 검출된 최대 score정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    #M_max_scores.append([0]*len(temp))
    
    
    
############################# !!!!!!!!!!!!!!!!!!!!!!!!!!! ############################# 
###테스트 결과 출력
def get_detection_info():
    list_not_detected_group_index = []  # 검출되지 않은 그룹의 index 목록(annotation 일치 조건 제외)
    list_not_detected_group_index_by_annotation = [] # annotation 일치 조건으로 검출되지 않은 그룹의 index 목록
    total_not_detected_file = 0 #검출되지 않은 전체 테스트 파일의 수    
    i_group_member = 0

    print('------------------------------------------------ Result info --------------------------------------------------------')
    #print('\n')    
    
    for idx_x, row in enumerate(groupby_detected_number_for_files):
        i_detections = 0
        for idx_y, elem in enumerate(row):
            i_detections += elem
            if elem == 0:
                total_not_detected_file += 1
                
        if i_detections == 0:
            list_not_detected_group_index.append(idx_x)
            i_group_member += len(row)
            #print('({})set have no detection!.'.format(groupby_key_list[idx_x]))

    #print('\n')

    #1st print type
    #print('Total Number of abnormal case not detected :::::::::: {}'.format(total_not_detected_file))
    #print('Number of abnormal case set not detected::::::: {}({} files)'.format(len(list_not_detected_group_index), i_group_member))

    #2nd print type
    #print('{}/{} files not detected.'.format(total_not_detected_file,len(test_file_list)))
    #print('{}({} files)/{} sets not detected.'.format(len(list_not_detected_group_index), i_group_member, len(groupby_key_list)))

    #3rd print type
    print('{} files'.format(total_not_detected_file))
    print('{}({} files) sets  not detected.'.format(len(list_not_detected_group_index), i_group_member))
    #print('\n')
    if 'normal' in DATA_TO_TEST:
        print('{} abnormal detected.(from {} normal files).'.format(number_abnormal_detect_from_normal,len(test_file_list)))
    
    # annotation 검출 정보
    print('\n')    
    
    i_group_member = 0
    num_anno_file = 0
    num_anno_mismatch_detect = 0
    #그룹(set)별 annotation match 검출 기준으로 
    for anno_idx_x, anno_row in enumerate(groupby_detected_info_for_anno_files):
        i_detections = 0
        for anno_idx_y, anno_elem in enumerate(anno_row):
            i_detections += anno_elem
            if anno_elem > 0:
                num_anno_file += 1
                if anno_elem < 100:
                    num_anno_mismatch_detect += 1
                   #print(groupby_file_list[anno_idx_x][anno_idx_y])
                    
        if i_detections < 100:
            list_not_detected_group_index_by_annotation.append(anno_idx_x)
            i_group_member += len(anno_row)
            #print('({})set not detected!.'.format(groupby_key_list[anno_idx_x]))
            
    print('{}/{} annotations not detected.'.format(num_anno_mismatch_detect, num_anno_file))    
    print('{}({} files)/{} sets not detected.'.format(len(list_not_detected_group_index_by_annotation), i_group_member, len(groupby_key_list)))
    
    print('---------------------------------------------------------------------------------------------------------------------')

###정보 초기화(일부 list 정보에 대한 초기화)
def init_info():
    for idx_i, row in enumerate(groupby_file_list):
        for idx_j, col in enumerate(row):
            groupby_detected_number_for_files[idx_i][idx_j] = 0
            groupby_detected_info_for_anno_files[idx_i][idx_j] = 0
            #M_max_scores[idx_i][idx_j] = 0
            
    set_anno_file_index()
    number_abnormal_detect_from_normal = 0
    
            
###그룹화된 테스트 파일 구조에서 annotation 정보가 있는 파일의 위치를 annotation dict 정보에 set
def set_anno_file_index():
    for ix, row in enumerate(groupby_file_list):
        for iy, elem in enumerate(row):
            if elem in annotation_dict:
                groupby_detected_info_for_anno_files[ix][iy] = 1
                annotation_dict[elem] = [ix, iy]
                
#set_anno_file_index()
#print(len(annotation_dict))
#print(annotation_dict)

## Run detection

In [ ]:
set_anno_file_index()

for SCORE_VALUE in SCORE_VALUE_LIST:
    print('****************{}*******************'.format(MODEL_ENV))
    print('*      Detecting with score {}           *'.format(SCORE_VALUE))
    print('*******************************************')
    
    model_count = 0;
    for model_path in model_list:
        #ensemble 테스트가 아닐경우 각 model별 result folder를 만들어 결과(file별 detection 표시)를 저장
        #ensemble일 경우는 ensemble folder에 최종 결과를 저장
        if not MODEL_ENV == 'ensemble': #ensemble 테스트가 아닐경우        
            result_save_path = os.path.join(result_save_subpath, 'score'+str(SCORE_VALUE), model_path.split('/')[-1].split('.')[-2])
            if not os.path.exists(result_save_path):
                os.makedirs(result_save_path)        

        model_count += 1
        number_of_proper_detection = 0 #검출된 수

        if MODEL_ENV == 'ensemble': #ensemble 테스트        
            print('\n{} model {}'.format(ordinal(model_count), model_path.split('/')[-1]))
        else:    
            print('\n{}. Load model {}'.format(model_count, model_path.split('/')[-1]))
            
        # Load RetinaNet Model
        model = keras.models.load_model(model_path, custom_objects=custom_objects)
        
        model2 = keras.models.load_model(model_list[1], custom_objects=custom_objects)
        
        model3 = keras.models.load_model(model_list[2], custom_objects=custom_objects)        
        

        img_index_list = not_detected_img_index_list.copy() #검출이 안된 파일 목록, 처음에는 전체 파일 목록이 대상이 됨.
        num_of_not_detected_files = len(img_index_list)
        print('{} test files.'.format(num_of_not_detected_files))
        
        print("start.")
        start = time.time()
        #print('0/{} completed.\r'.format(num_of_test_files), end='')
        #for index in img_index_list:
        for index_count, index in enumerate(img_index_list):
            b_anno_match = False
            b_is_annotation = False
            b_save_pass = False

            # load image
            image, image_path = val_generator.load_image(index, get_image_path=True)
            #print('image :::::::::: {}'.format(image_path))
            image_name_split = image_path.split('/')[-1].split('-')

            # copy to draw on
            draw = image.copy()
            draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

            # preprocess image for network
            image = val_generator.preprocess_image(image)
            image, scale = val_generator.resize_image(image)
            annotations = val_generator.load_annotations(index)

            # process image
            _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))

            # compute predicted labels and scores
            predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
            scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

            # correct for image scale
            detections[0, :, :4] /= scale

            # for visualize annotations if imagae include annotation
            if len(annotations) > 0: 
                b_is_annotation = True #현재 파일이 annotation 정보를 가지고 있음
                anno_center = []
                for annotation in annotations:
                    label = int(annotation[4])
                    b = annotation[:4].astype(int)

                    anno_center.append((b[0]+b[2])/2)
                    anno_center.append((b[1]+b[3])/2)

                    cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (50, 50, 255), 2)
                    caption = "{}".format(val_generator.label_to_name(label))
                    cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
                    cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
            
            number_of_proper_detection = 0 #현재 파일에서 detect된 수
            '''
            M_max_score = 0 
            N_max_score = 0
            '''
            for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
                '''        
                if label == 0:
                    if M_max_score < score:
                        M_max_score = score    
                else:
                    if N_max_score < score:
                        N_max_score = score
                '''

                if score < SCORE_VALUE or label != 0:
                    continue

                number_of_proper_detection += 1
                b = detections[0, idx, :4].astype(int)

                #cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
                cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 2)
                caption = "{} {:.3f}".format(val_generator.label_to_name(label), score)
                cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
                cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
                
                if b_is_annotation and not b_anno_match and len(anno_center) > 0:
                    if b[0]<anno_center[0]<b[2] and b[1]<anno_center[1]<b[3]:
                        b_anno_match = True # annotation과 매칭되는 검출정보가 있음
                    
            '''
            # 테스트 파일 그룹 목록에서 현재 파일의 위치를 구함    
            img_indics = find(groupby_file_list, image_path)
            if img_indics[0] >= 0:
                # 검출된 정보 중 Max 스코어 값을 파일 위치와 매칭하여 저장
                M_max_scores[img_indics[0]][img_indics[1]] = M_max_score 
            '''
                
            #이미지에서 (조건에 부합되게)detect된 것이 있을 경우
            if number_of_proper_detection > 0:
                if 'normal' in image_path:
                    number_abnormal_detect_from_normal += 1            

                ##################### 그룹화된 테스트 파일 정보와 동일한 구조의 detection 정보 목록에 반영 ##############
                # 테스트 파일 그룹 목록에서 현재 파일의 위치를 구함    
                img_indics = find(groupby_file_list, image_path)
                if len(img_indics) > 0:
                    #groupby_detected_number_for_files[img_indics[0]][img_indics[1]] += number_of_proper_detection
                    groupby_detected_number_for_files[img_indics[0]][img_indics[1]] = 1

                 # (조건에 부합되게) 검출된것 중에서 annotation과 일치가 될 경우
                if b_is_annotation and b_anno_match:
                    anno_file_pos = annotation_dict[image_path]
                    groupby_detected_info_for_anno_files[anno_file_pos[0]][anno_file_pos[1]] = 100
                    
                ##Ensemble일경우 테스트 파일 목록에서 검출된 파일 목록을 삭제(다음 모델 테스트에서 제외)
                #단, annotation이 있는 경우에는 annotation match 검출일때만 목록에서 제거, 다음 모델 테스트로 넘김
                if MODEL_ENV == 'ensemble': #ensemble 테스트일 경우
                    if not b_is_annotation or b_anno_match: 
                        not_detected_img_index_list.remove(index) 

                        
            ############검출된 정보 및 annotation을 이미지에 표시, 저장################
            ############ 전체 파일 저장할 경우는 #1,#3번###############
            ############ annotation이 있는 파일만 저장할 경우는 #2,#2번###############
                #1. 파일저장, annotation 유무에 관계없이, 검출영역 정보 표시
                # 기존에 이미 저장된 파일이 있을 경우는 update됨 
                #'''
                cv2.imwrite(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), draw)
                b_save_pass = True
                
                #'''
                #2. annotation 정보를 가지고 있는 파일만 저장
                '''
                if b_is_annotation:
                    cv2.imwrite(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), draw)
                    b_save_pass = True
                '''
            
            if not b_save_pass:
                b_save_pass = os.path.isfile(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])))
                
            #3. 모든 파일저장, annotation 유무, 검출영역 존재 여부에 관계없이
            # 단, 검출영역이 존재하는 파일은 위에서 저장 했으므로 검출안된 정보로 업데이트 될 수 있어서
            # 이미 저장된 파일이 경우는 pass
            #'''
            if not b_save_pass:
                cv2.imwrite(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), draw)
                
            #'''    
            #4. annotation 정보를 가지고 있는 파일만 저장, 검출 결과도 함께 표시
            '''
            if not b_save_pass and b_is_annotation:
                cv2.imwrite(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), draw)
                        
            #annotation 미검출(검출결과가 annotation 일치 조건을 만족하지 못함)된 파일만 따로 저장
            if MODEL_ENV == 'ensemble': #ensemble 테스트일 경우
                #맨 마지막 모델에서 annotation mismatch된 파일만 저장
                #각 모델에서 검출 되었더라도 annotation mismatch된 목록은 리스트에서 remove하지 않고 다음 모델로 넘겼기 때문에
                #마지막 모델 테스트 시 annotation 정보를 가지면서 mismatch된 파일만 따로 저장
                if model_count == len(model_list) and b_is_annotation and not b_anno_match:
                    # 앞서 저장된 파일을 따로 rename하여 복사
                    shutil.copy2(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), os.path.join(result_save_path, ('-'.join(image_path.split('/')[-2:])).split('.')[0]+'_miss'+'.jpg'))
                    #cv2.imwrite(os.path.join(result_save_path, ('-'.join(image_path.split('/')[-2:])).split('.')[0]+'_miss'+'.jpg'), draw)
                    #print(os.path.join(result_save_path, ('-'.join(image_path.split('/')[-2:])).split('.')[0]+'_miss'+'.jpg'))
            '''
            #print('{}/{} completed.\r'.format(index+1,num_of_test_files), end='')
            print('{}/{} completed.\r'.format(index_count+1,num_of_not_detected_files), end='')

        print("end({:.2f} minutes elapsed).".format((time.time() - start)/60))

        if not MODEL_ENV == 'ensemble': #ensemble 테스트가 아닐경우
            get_detection_info() #detection 결과 출력
            init_info() #next model로 변경을 위해 현재 model의 detection 정보를 초기화
        
        # release the memory
        del model
        keras.backend.clear_session()
        
    if MODEL_ENV == 'ensemble': #ensemble 테스트가 아닐경우        
        get_detection_info() #detection 결과 출력
    
    print('\n')        

In [ ]:
print('Finished!')